### 1. Read undrafted player's college data

In [1]:
import pandas as pd
from pathlib import Path

current_dir = Path.cwd()
undrafted_df = pd.read_csv(current_dir.parent/ "data"/ "raw" /"college_undrafted" / "undrafted_college_stats.csv")
undrafted_df.head()

,PLAYER_ID,FIRST_NAME,LAST_NAME,cbb_url,Season,Team,Conf,Class,Pos,G,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards
0,2090,Justin,Love,https://www.sports-reference.com/cbb/players/j...,1999-00,Saint Louis,CUSA,SR,G,33.0,...,49.0,112.0,161,83,49,8,97,83,602.0,NaN
1,12144,Brandon,Kurtz,https://www.sports-reference.com/cbb/players/b...,1999-00,Tulsa,WAC,SR,C,37.0,...,87.0,171.0,258,59,44,47,83,120,414.0,NaN
2,12143,Kenyon,Jones,https://www.sports-reference.com/cbb/players/k...,1999-00,San Francisco,WCC,SR,C,28.0,...,93,160,253,24,25,40,71,90,462.0,NaN
3,12142,Nate,Johnson,https://www.sports-reference.com/cbb/players/n...,1998-99,Buffalo,MAC,SR,C,29.0,...,51.0,97.0,148.0,15.0,11.0,5.0,48.0,55.0,103.0,NaN
4,12141,Jacob,Jaacks,https://www.sports-reference.com/cbb/players/j...,1999-00,Iowa,Big Ten,SR,C,30.0,...,55,165,220,29,10,13,75,118,367.0,NaN


### 2. Strip all the column names

In [2]:
# Strip white spaces from column names
undrafted_df.columns = undrafted_df.columns.str.strip()

# Strip white spaces from column values
undrafted_df = undrafted_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

/var/folders/b9/7xp5dt9d5nd0hct357z94wgh0000gn/T/ipykernel_36710/3491261490.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  undrafted_df = undrafted_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


### 3. Preprocess Season 1999-00 to 1999

In [3]:
# Change Season
undrafted_df["SEASON"] = undrafted_df["Season"].str[:4].astype(int) + 1

# Fix edge case: map 2026 → 2025
undrafted_df.loc[undrafted_df["SEASON"] == 2026, "SEASON"] = 2025


### 3. Cross Validation with NBA Combine Dataset

In [4]:
import pandas as pd
from pathlib import Path

current_dir = Path.cwd()
nba_combine_df = pd.read_csv(current_dir.parent/ "data"/ "cleaned" /"draft_combine.csv")
nba_combine_df.head()

,PERSON_ID,PLAYER_NAME,SEASON,ROUND_NUMBER,OVERALL_PICK,TEAM_NAME,ORGANIZATION,ORGANIZATION_TYPE,PLAYER_PROFILE_FLAG,POSITION,...,OFF_DRIB_FIFTEEN_BREAK_LEFT,OFF_DRIB_FIFTEEN_TOP_KEY,OFF_DRIB_FIFTEEN_BREAK_RIGHT,OFF_DRIB_COLLEGE_BREAK_LEFT,OFF_DRIB_COLLEGE_TOP_KEY,OFF_DRIB_COLLEGE_BREAK_RIGHT,ON_MOVE_FIFTEEN,ON_MOVE_COLLEGE,BMI,VERTICAL_RATIO
0,2030,Kenyon Martin,2000,1,1,Nets,Cincinnati,College/University,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2031,Stromile Swift,2000,1,2,Grizzlies,Louisiana State,College/University,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2032,Darius Miles,2000,1,3,Clippers,East St. Louis,High School,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2033,Marcus Fizer,2000,1,4,Bulls,Iowa State,College/University,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2034,Mike Miller,2000,1,5,Magic,Florida,College/University,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import pandas as pd

# --- Undrafted: build full name ---
undrafted_df = undrafted_df.copy()
undrafted_df["FULL_NAME"] = (
    undrafted_df["FIRST_NAME"].str.strip() + " " +
    undrafted_df["LAST_NAME"].str.strip()
).str.lower()

# --- Combine: normalize player name ---
nba_combine_df = nba_combine_df.copy()
nba_combine_df["FULL_NAME"] = (
    nba_combine_df["PLAYER_NAME"]
    .str.strip()
    .str.lower()
)

# Ensure SEASON types match
undrafted_df["SEASON"] = undrafted_df["SEASON"].astype(str)
nba_combine_df["SEASON"] = nba_combine_df["SEASON"].astype(str)


In [6]:
merged = undrafted_df.merge(
    nba_combine_df[["FULL_NAME", "SEASON"]],
    on=["FULL_NAME", "SEASON"],
    how="left",
    indicator=True
)

In [7]:
undrafted_filtered_df = merged[merged["_merge"] == "left_only"] \
    .drop(columns=["_merge", "FULL_NAME"])


In [8]:
print("Before:", len(undrafted_df))
print("After :", len(undrafted_filtered_df))


Before: 582
After : 559


In [10]:
undrafted_df = undrafted_filtered_df.copy()

### 4. Rename columns

In [11]:
undrafted_df["player_name"] = undrafted_df["FIRST_NAME"] + " " + undrafted_df["LAST_NAME"]

rename_map = {
    "FG": "Totals_FG",
    "FT": "Totals_FT",
    "TRB": "Totals_TRB",
    "BLK": "Totals_BLK",
    "STL": "Totals_STL",
    "TOV": "Totals_TOV",
    "PF" : "Totals_PF",
    "FG%": "Shooting_FG%",
}

undrafted_df = undrafted_df.rename(columns=rename_map)

final_cols = [
    "player_name", "SEASON",
    "Totals_FG", "Totals_FT", "Totals_TRB",
    "Totals_BLK", "Totals_STL", "Totals_TOV",
    "Totals_PF", "Shooting_FG%", "MP"
]

undrafted_filtered = undrafted_df[final_cols]
undrafted_filtered.head()

,player_name,SEASON,Totals_FG,Totals_FT,Totals_TRB,Totals_BLK,Totals_STL,Totals_TOV,Totals_PF,Shooting_FG%,MP
0,Justin Love,2000,190.0,151.0,161,8,49,97,83,0.438,1030
1,Brandon Kurtz,2000,155.0,101.0,258,47,44,83,120,0.513,998
2,Kenyon Jones,2000,155.0,148.0,253,40,25,71,90,.583,730
3,Nate Johnson,1999,36.0,31.0,148.0,5.0,11.0,48.0,55.0,0.4,581.0
4,Jacob Jaacks,2000,123.0,100.0,220,13,10,75,118,0.468,822


### 5. Read drafted player's college data

In [12]:
import pandas as pd
from pathlib import Path

current_dir = Path.cwd()
drafted_df = pd.read_csv(current_dir.parent/ "data" / "cleaned" / "college_drafted" / "college_drafted_selected_features.csv")
drafted_df.head()

,player_name,OVERALL_PICK,SEASON,Totals_FG,Totals_FT,Totals_TRB,Totals_BLK,Totals_STL,Totals_TOV,Totals_PF,Shooting_FG%,MP,Age
0,Kenyon Martin,1,2000,221,141,300,107.0,43.0,56.0,71.0,0.568,909.0,22.0
1,Stromile Swift,2,2000,208,127,279,95.0,50.0,80.0,88.0,0.608,1013.0,20.0
2,Marcus Fizer,4,2000,327,175,285,39.0,29.0,77.0,103.0,0.582,1243.0,21.0
3,Mike Miller,5,2000,175,124,243,15.0,46.0,71.0,70.0,0.476,1058.0,19.0
4,DerMarr Johnson,6,2000,140,70,123,30.0,31.0,46.0,64.0,0.478,879.0,19.0


### 6. Impute Missing Values

In [13]:
undrafted_filtered["OVERALL_PICK"] = 100

feature_cols = [
    "SEASON",
    "OVERALL_PICK",
    "Totals_FG", "Totals_FT", "Totals_TRB",
    "Totals_BLK", "Totals_STL", "Totals_TOV",
    "Totals_PF", "Shooting_FG%", "MP"
]
undrafted_filtered.head()

num_cols = [
    "Totals_FG", "Totals_FT", "Totals_TRB",
    "Totals_BLK", "Totals_STL", "Totals_TOV",
    "Totals_PF", "Shooting_FG%", "MP",
]

for col in num_cols:
    drafted_df[col] = pd.to_numeric(drafted_df[col], errors="coerce")
    undrafted_filtered[col] = pd.to_numeric(undrafted_filtered[col], errors="coerce")



/var/folders/b9/7xp5dt9d5nd0hct357z94wgh0000gn/T/ipykernel_36710/2281794846.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  undrafted_filtered["OVERALL_PICK"] = 100
/var/folders/b9/7xp5dt9d5nd0hct357z94wgh0000gn/T/ipykernel_36710/2281794846.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  undrafted_filtered[col] = pd.to_numeric(undrafted_filtered[col], errors="coerce")


In [14]:
from sklearn.impute import KNNImputer
import pandas as pd

num_cols = [
    "Totals_FG", "Totals_FT", "Totals_TRB",
    "Totals_BLK", "Totals_STL", "Totals_TOV",
    "Totals_PF", "Shooting_FG%", "MP"
]

imputer = KNNImputer(n_neighbors=5)

undrafted_imputed = undrafted_filtered.copy()
undrafted_imputed[num_cols] = imputer.fit_transform(undrafted_filtered[num_cols])

/Users/tenzin/Desktop/mycodes/nba-draft-ranker/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tenzin/Desktop/mycodes/nba-draft-ranker/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/tenzin/Desktop/mycodes/nba-draft-ranker/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


### 7. Merge data

In [15]:
drafted_clean   = drafted_df[feature_cols]
undrafted_clean = undrafted_imputed[feature_cols]


merged_df = pd.concat([drafted_clean, undrafted_clean], ignore_index=True)
merged_df.sort_values(by=["SEASON", "OVERALL_PICK"], inplace=True)

merged_df.to_csv(current_dir.parent / "data" /"cleaned"/  "college_undrafted.csv", index=False)

In [16]:
merged_df["OVERALL_PICK"].eq(100).value_counts().rename(
    {True: "Undrafted", False: "Drafted"}
)


OVERALL_PICK
Drafted      1210
Undrafted     559
Name: count, dtype: int64